In [ ]:
import numpy as np
import sympy as sy
sy.init_printing() 

In [ ]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # display multiple lines

In [ ]:
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sy.Number)})

# <font face="gotham" color="purple"> Operaciones matriciales</font>

Las operaciones de _suma_ de matrices son sencillas:
1. $A+ B= B+ A$
2. $(A+B)+ C=A+(B+C)$
3. $c(A+B)=cA+cB$
4. $(c+d)A=cA+c{D}$
5. $c(dA)=(cd)A$
6. $A+{0}=A$, donde ${0}$ es la matriz nula
7. Para cualquier $A$, existe un $- A$, tal que $ A+(- A)=0$.

Son tan obvias como parecen, por lo que no se proporcionan pruebas. Y las propiedades de _multiplicación_ de matrices son:
1. $ A({BC})=({AB}) C$
2. $c({AB})=(cA)B=A(cB)$
3. $A(B+ C)={AB}+{AC}$
4. $(B+C)A={BA}+{CA}$

Ten en cuenta que necesitamos diferenciar entre dos tipos de multiplicación: _multiplicación de Hadamard_, denotada como $A \odot B$ (multiplicación elemento a elemento), y _multiplicación de matrices_, denotada simplemente como $AB$.

In [ ]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

In [ ]:
A*B # this is Hadamard elementwise product

In [ ]:
A@B # this is matrix product

Mostremos explícitamente la regla de multiplicación de matrices para cada elemento:

In [ ]:
np.sum(A[0,:]*B[:,0]) # element at (1, 1)
np.sum(A[1,:]*B[:,0]) # element at (2, 1)
np.sum(A[0,:]*B[:,1]) # element at (1, 2)
np.sum(A[1,:]*B[:,1]) # element at (2, 2)

## <font face="gotham" color="purple"> Demostración de SymPy: Suma </font>

Definamos todas las letras como símbolos en caso de que necesitemos usarlas repetidamente. Con esta biblioteca, podemos realizar cálculos analíticamente, lo que la convierte en una herramienta valiosa para aprender álgebra lineal.

In [ ]:
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z = sy.symbols('a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z', real = True)

In [ ]:
A = sy.Matrix([[a, b, c], [d, e, f]])
A + A
A - A

In [ ]:
B = sy.Matrix([[g, h, i], [j, k, l]])
A + B
A - B

## <font face="gotham" color="purple"> Demostración de SymPy: Multiplicación </font>

Las reglas de multiplicación de matrices se pueden entender claramente usando símbolos.

In [ ]:
A = sy.Matrix([[a, b, c], [d, e, f]])
B = sy.Matrix([[g, h, i], [j, k, l], [m, n, o]])
A
B

In [ ]:
AB = A*B; AB

## <font face="gotham" color="purple"> Conmutabilidad </font>

La multiplicación de matrices generalmente no conmuta, lo que significa ${AB} \neq {BA}$. Por ejemplo, considera las matrices $A$ y $B$:

In [ ]:
A = sy.Matrix([[3, 4], [7, 8]])
B = sy.Matrix([[5, 3], [2, 1]])
A*B
B*A

¿Cómo encontramos una matriz conmutable? Intentemos averiguarlo analíticamente

In [ ]:
A = sy.Matrix([[a, b], [c, d]])
B = sy.Matrix([[e, f], [g, h]])
A*B
B*A

Para mostrar ${AB} = {BA}$, necesitamos probar ${AB} - {BA} = 0$

In [ ]:
M = A*B - B*A; M

Eso nos da un sistema de ecuaciones lineales
$$
\begin{align}
b g - c f&=0 \\
 a f - b e + b h - d f&=0\\
- a g + c e - c h + d g&=0 \\
- b g + c f&=0
\end{align}
$$

Para resolverlo, podemos usar el método de eliminación de Gauss-Jordan. Si tratamos $a, b, c, d$ como coeficientes del sistema, podemos extraer una matriz aumentada.
$$
\begin{align*}
b g-c f=0 \quad &\Rightarrow-c f+b g+0 e+0 h=0\\
a f-b e+b h-d f=0 \quad &\Rightarrow(a-d) f+0 g-b e+b h=0\\
-a g+c e-c h+d g=0 \quad &\Rightarrow 0 f+(d-a) g+c e-c h=0\\
-b g+c f=0 \quad &\Rightarrow c f-b g+0 e+0 h=0
\end{align*}
$$

Así, la matriz aumentada toma la forma
$$
\begin{equation}
\left[\begin{array}{cccc:c}
-c & b & 0 & 0 & 0 \\
a-d & 0 & -b & b & 0 \\
0 & d-a & c & -c & 0 \\
c & -b & 0 & 0 & 0
\end{array}\right]
\end{equation}
$$

In [ ]:
A_aug = sy.Matrix([[-c, b, 0, 0], [a-d,0, -b, b], [0, d-a, c, -c], [c, -b, 0, 0]]); A_aug

Realiza la eliminación de _Gauss-Jordan_ hasta la forma escalonada reducida por filas.

In [ ]:
A_aug.rref()

La solución general es 
$$
\begin{equation}
\left(\begin{array}{l}
e \\
f \\
g \\
h
\end{array}\right)=c_1\left(\begin{array}{c}
\frac{b}{a-d} \\
\frac{b c}{a b-b d} \\
1 \\
0
\end{array}\right)+c_2\left(\begin{array}{c}
-\frac{b}{a-d} \\
-\frac{c}{a-d} \\
0 \\
1
\end{array}\right)
\end{equation}
$$

In [ ]:
import sympy as sp

# Define symbolic entries for A (2x2 matrix)
a, b, c, d = sp.symbols('a b c d')

# Define symbolic entries for B (2x2 matrix)
e, f, g, h = sp.symbols('e f g h')

# Define matrices A and B
A = sp.Matrix([[a, b], [c, d]])
B = sp.Matrix([[e, f], [g, h]])

# Compute AB and BA
AB = A * B
BA = B * A

# Set up equations AB = BA
equations = [
    sp.Eq(AB[i, j], BA[i, j]) for i in range(2) for j in range(2)
]

# Solve the system of equations
solution = sp.solve(equations, (e, f, g, h))

# Print the general solutions
print("Solution for B elements:")
for sol in solution:
    print(f"{sol}: {solution[sol]}")

# To express the solution in a parameterized form, let's substitute specific values
# Define parameters c1 and c2
c1, c2 = sp.symbols('c1 c2')

# Define the parameterized solution
parametric_solution = {
    e: solution[e].subs({g: c1, h: c2}),
    f: solution[f].subs({g: c1, h: c2}),
    g: c1,
    h: c2
}

# Print the parameterized solution in LaTeX format
print("\nParameterized solution in LaTeX format:")
for sol in parametric_solution:
    print(f"{sp.latex(sol)}: {sp.latex(parametric_solution[sol])}")


$$
e: c_{2} + \frac{c_{1} \left(a - d\right)}{c}\\
f: \frac{b c_{1}}{c}\\
g: c_{1}\\
h: c_{2}\\
$$

# <font face="gotham" color="purple"> Transposición de matrices </font>

La matriz $A_{n\times m}$ y su transpuesta es 

In [ ]:
A = sy.Matrix([[1, 2, 3], [4, 5, 6]]); A
A.transpose()

Las propiedades de la transposición son
1. $(A^T)^T$
2. $(A+B)^T=A^T+B^T$
3. $(cA)^T=cA^T$
4. $(AB)^T=B^TA^T$

Podemos demostrar por qué la última propiedad se cumple con SymPy, definimos $A$ y $B$, los multiplicamos y luego transponemos, eso significa $(AB)^T$

In [ ]:
A = sy.Matrix([[a, b], [c, d], [e, f]])
B = sy.Matrix([[g, h, i], [j, k, l]])
AB = A*B
AB_t = AB.transpose(); AB_t

Transponga $A$ y $B$, luego multiplíquelos, es decir $B^TA^T$

In [ ]:
B_t_A_t = B.transpose()*A.transpose()
B_t_A_t

Check if they are equal

In [ ]:
AB_t == B_t_A_t

# <font face="gotham" color="purple"> Matrices de identidad </font>

Esta es una matriz identidad $I_5$, solo $1$ en la diagonal principal, todos los elementos restantes son $0$.

In [ ]:
sy.eye(5)

Propiedades de la matriz de identidad:

$$
AI=IA = A
$$

Generemos $ I $ y $ A $ y demostremos si se cumple

In [ ]:
I = np.eye(5); I

In [ ]:
A = np.around(np.random.rand(5, 5)*100); A # generate a random matrix

Comprueba si son iguales

In [ ]:
(A@I == I@A).all()

# <font face="gotham" color="purple"> Matriz elemental </font>

Una _matriz elemental_ es una matriz que se puede obtener mediante una sola operación de fila elemental sobre una matriz identidad. Por ejemplo:

$$
\left[
\begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}
\right]
\begin{array}{c}
R_1 \leftrightarrow R_2 \\
~ \\
~
\end{array}
\qquad \Longrightarrow \qquad
\left[
\begin{array}{ccc}
0 & 1 & 0 \\
1 & 0 & 0 \\
0 & 0 & 1
\end{array}
\right]
$$

Donde $R_1 \leftrightarrow R_2$ significa intercambiar las filas 1 y 2, denotamos la matriz transformada como $E$. Luego, multiplicar $E$ por la izquierda sobre una matriz $A$ realizará exactamente la misma operación de fila.

Primero, genere la matriz $A$.

In [ ]:
A = sy.randMatrix(3, percent = 80); A # generate a random matrix with 80% of entries being nonzero

Crea una matriz elemental con $R_1\leftrightarrow R_2$

In [ ]:
E = sy.Matrix([[0, 1, 0], [1, 0, 0], [0, 0, 1]]);E

Observe que al multiplicar por la izquierda $E$ sobre $A$, la matriz $A$ también intercambia las filas 1 y 2.

In [ ]:
E*A

Sumar un múltiplo de una fila a otra fila en la matriz identidad también nos da una matriz elemental.

$$

\left[
\begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}
\right]\\

\begin{array}{c}
R_3-7R_1
\end{array}
\longrightarrow
\left[
\begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & 0 \\
-7 & 0 & 1
\end{array}
\right]

$$

Verifiquemos con SymPy.

In [ ]:
A = sy.randMatrix(3, percent = 80); A
E = sy.Matrix([[1, 0, 0], [0, 1, 0], [-7, 0, 1]]); E

We will see the $R_3-7R_1$ takes places on $A$

In [ ]:
E*A

We can also reproduce this by explicit row operation on $ A$.

In [ ]:
EA = sy.matrices.MatrixBase.copy(A)
EA[2,:]=-7*EA[0,:]+EA[2,:]
EA

In the next section, we will refresh an important conclusion: an _invertible matrix_ is a product of a series of elementary matrices.

# <font face="gotham" color="purple"> Inverse Matrices </font>

If ${AB}={BA}=\mathbf{I}$, $ B$ is called the inverse of matrix $  A$, denoted as $ B=  A^{-1}$.


NumPy has convenient function ```np.linalg.inv()``` for computing inverse matrices. Generate $ A$

In [ ]:
A = np.round(10*np.random.randn(5,5)); A

In [ ]:
Ainv = np.linalg.inv(A); Ainv

Verify if they are truly inverse of each other

In [ ]:
A@Ainv

The ```-0.``` means there are more digits after point, but omitted here.

## <font face="gotham" color="purple"> Gauss-Jordan Elimination Method for Matrix Inversion</font>

A convenient way to calculate the inverse of a matrix is to construct an augmented matrix $[A \,|\, \mathbf{I}]$. Then, multiply a series of elementary matrices $E$ (representing elementary row operations) until matrix $A$ is in row-reduced form. If $A$ is of full rank, this process will transform $A$ into an identity matrix $\mathbf{I}$. Consequently, the identity matrix on the right-hand side of the augmented matrix will be converted into $A^{-1}$ automatically.

We can demonstrate this using SymPy's `.rref()` function on the augmented matrix $[A \,|\, \mathbf{I}]$.

In [ ]:
AI = np.hstack((A, I)) # stack the matrix A and I horizontally
AI = sy.Matrix(AI); AI

In [ ]:
AI_rref = AI.rref(); AI_rref

Extract the RHS block, this is the $A^{-1}$.

In [ ]:
Ainv = AI_rref[0][:,5:];Ainv # extract the RHS block

I wrote a function to round the float numbers to the $4$-th digits, on the top of this file, just for sake of readability.

In [ ]:
round_expr(Ainv, 4) 

We can verify if $AA^{-1}=\mathbf{I}$

In [ ]:
A = sy.Matrix(A)
round_expr(A*Ainv, 4) 

We got $\mathbf{I}$, which means the RHS block is indeed $A^{-1}$.

## <font face="gotham" color="purple"> An Example of Existence of Inverse </font>

Determine the values of $\lambda$ such that the matrix
$$A=\left[ \begin{matrix}3 &\lambda &1\cr 2 & -1 & 6\cr 1 & 9 & 4\end{matrix}\right]$$
is not invertible.

Still,we are using SymPy to solve the problem.

In [ ]:
lamb = sy.symbols('lamda') # SymPy will automatically render into LaTeX greek letters
A = np.array([[3, lamb, 1], [2, -1, 6], [1, 9, 4]])
I = np.eye(3); A

Form the augmented matrix.

In [ ]:
AI = np.hstack((A, I))
AI = sy.Matrix(AI); AI

In [ ]:
AI_rref = AI.rref()
AI_rref

To make the matrix $A$ invertible we notice that is multiple conditions to be satisfied, (in the denominators):
$$
\begin{align*}
-6\lambda -465 &\neq0\\
4 \lambda + 310 &\neq 0\\
2 \lambda + 155 &\neq 0
\end{align*}
$$
However, they are actually one condition, because they are multiples of each other.

Solve for $\lambda$'s.

In [ ]:
sy.solvers.solve(-6*lamb-465, lamb)

So this is the $\lambda$ that makes the matrix invertible. Let's test this with the _determinant_. If $|A| = 0$, then the matrix is not invertible, so plug in $\lambda$ back in $A$ then calculate determinant. Don't worry about determinants; we will revisit this topic later.

In [ ]:
A = np.array([[3, -155/2, 1], [2, -1, 6], [1, 9, 4]])
np.linalg.det(A)

The $| A|$ is $0$. 

So we found that one condition, as long as $\lambda \neq -\frac{155}{2}$, the matrix $A$ is invertible.

## <font face="gotham" color="purple"> Properties of Inverse Matrices </font>

1. If $A$ and $B$ are both invertible, then $(AB)^{-1}=B^{-1}A^{-1}$.
2. If $A$ is invertible, then $(A^T)^{-1}=(A^{-1})^T$.
3. If $A$ and $B$ are both invertible and symmetric such that $AB=BA$, then $A^{-1}B$ is symmetric.

The _first property_ is straightforward
$$
\begin{align}
ABB^{-1}A^{-1}=AIA^{-1}=I=AB(AB)^{-1}
\end{align}
$$

The trick of _second property_ is to show that
$$
A^T(A^{-1})^T = I
$$
We can use the property of transpose
$$
A^T(A^{-1})^T=(A^{-1}A)^T = I^T = I
$$

The _third property_ is to show
$$
A^{-1}B = (A^{-1}B)^T
$$
Again use the property of transpose
$$
(A^{-1}B)^{T}=B^T(A^{-1})^T=B(A^T)^{-1}=BA^{-1}
$$
We use the $AB = BA$ condition to proceed
\begin{align*}
AB&=BA\\
A^{-1}ABA^{-1}&=A^{-1}BAA^{-1}\\
BA^{-1}&=A^{-1}B
\end{align*}
The plug in the previous equation, we have
$$
(A^{-1}B)^{T}=BA^{-1}=A^{-1}B
$$